In [15]:



# import dependencies
import pandas as pd
import numpy as np
import sys
from datetime import datetime
import getpass

# # For MySQL
#from sqlalchemy import create_engine
#import pandas as pd
#import pymysql

# For PostgreSQL
from sqlalchemy import create_engine
import pandas as pd
import psycopg2

# # For SQL Server
# from sqlalchemy import create_engine
# import pandas as pd
# import pyodbc

In [16]:
!which python

/Users/admin/.pyenv/versions/etl-env/bin/python


In [ ]:
!pip install mysql-connector-python
! pip install google-cloud-storage 
username = getpass.getpass('Enter username:')
password = getpass.getpass('Enter password:')
public_ip_address = '35.239.201.185'
db_name = 'gmb'
test_db_name = 'gmb_test'

# Set your GCS bucket configurations
gcs_bucket_name = 'gmb-database-backup'
gcs_backup_blob_name = 'backup.sql'
# Path to store the temporary backup file
TEMP_BACKUP_PATH = '/tmp/backup.sql'

In [ ]:
import psycopg2

username = 'gmb'
password = getpass.getpass('Enter password:')
public_ip_address = '35.239.201.185'
db_name = 'gmb'

# Create a connection
conn = psycopg2.connect(
    dbname=db_name,
    user=username,
    password=password,
    host=public_ip_address
)

# Create a cursor
cur = conn.cursor()

In [ ]:
# Query to get all table names in the PostgreSQL database
query = """
SELECT tablename 
FROM pg_catalog.pg_tables 
WHERE schemaname != 'pg_catalog' AND 
      schemaname != 'information_schema';
"""

# Execute the query
cur.execute(query)

In [ ]:
tables = cur.fetchall()

# print the table names
for table in tables:
    print(table[0])  
